In [1]:
import whisper
import torch
import yt_dlp as youtube_dl
from pydub import AudioSegment
from IPython.display import Audio
from huggingface_hub import login

import os
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

device = "cuda" if torch.cuda.is_available() else "cpu"

/opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


# Helper functions

In [2]:
import yt_dlp

def get_youtube_video_title(url):
    ydl_opts = {}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(url, download=False)
        video_title = info_dict.get('title', None)
    return video_title

def get_youtube_video_metadata(video_url):
    ydl_opts = {
        'skip_download': True,  # Não baixa o vídeo
        'extract_flat': True,   # Não extrai streams de mídia, só metadados
    }
    
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(video_url, download=False)

    return info_dict


In [3]:
import re

def remove_special_characters(text):
    return re.sub(r'[^A-Za-z0-9\s]+', '', text)

dict_keys(['id', 'title', 'formats', 'thumbnails', 'thumbnail', 'description', 'channel_id', 'channel_url', 'duration', 'view_count', 'average_rating', 'age_limit', 'webpage_url', 'categories', 'tags', 'playable_in_embed', 'live_status', 'release_timestamp', '_format_sort_fields', 'automatic_captions', 'subtitles', 'comment_count', 'chapters', 'heatmap', 'like_count', 'channel', 'channel_follower_count', 'uploader', 'uploader_id', 'uploader_url', 'upload_date', 'timestamp', 'availability', 'original_url', 'webpage_url_basename', 'webpage_url_domain', 'extractor', 'extractor_key', 'playlist', 'playlist_index', 'display_id', 'fulltitle', 'duration_string', 'release_year', 'is_live', 'was_live', 'requested_subtitles', '_has_drm', 'epoch', 'requested_formats', 'format', 'format_id', 'ext', 'protocol', 'language', 'format_note', 'filesize_approx', 'tbr', 'width', 'height', 'resolution', 'fps', 'dynamic_range', 'vcodec', 'vbr', 'stretched_ratio', 'aspect_ratio', 'acodec', 'abr', 'asr', 'audi

# Fazendo download de vídeo do youtube

In [5]:
VIDEO_URL = 'https://www.youtube.com/watch?v=tvIzBouq6lk'
TRANSCRIPTION_LANGUAGE = "en"

#### Obtendo o nome do vídeo

In [6]:
video_metadata = get_youtube_video_metadata(VIDEO_URL)

[youtube] Extracting URL: https://www.youtube.com/watch?v=tvIzBouq6lk
[youtube] tvIzBouq6lk: Downloading webpage
[youtube] tvIzBouq6lk: Downloading ios player API JSON
[youtube] tvIzBouq6lk: Downloading web creator player API JSON
[youtube] tvIzBouq6lk: Downloading m3u8 information


In [7]:
video_metadata.keys()

dict_keys(['id', 'title', 'formats', 'thumbnails', 'thumbnail', 'description', 'channel_id', 'channel_url', 'duration', 'view_count', 'average_rating', 'age_limit', 'webpage_url', 'categories', 'tags', 'playable_in_embed', 'live_status', 'release_timestamp', '_format_sort_fields', 'automatic_captions', 'subtitles', 'comment_count', 'chapters', 'heatmap', 'like_count', 'channel', 'channel_follower_count', 'uploader', 'uploader_id', 'uploader_url', 'upload_date', 'timestamp', 'availability', 'original_url', 'webpage_url_basename', 'webpage_url_domain', 'extractor', 'extractor_key', 'playlist', 'playlist_index', 'display_id', 'fulltitle', 'duration_string', 'release_year', 'is_live', 'was_live', 'requested_subtitles', '_has_drm', 'epoch', 'requested_formats', 'format', 'format_id', 'ext', 'protocol', 'language', 'format_note', 'filesize_approx', 'tbr', 'width', 'height', 'resolution', 'fps', 'dynamic_range', 'vcodec', 'vbr', 'stretched_ratio', 'aspect_ratio', 'acodec', 'abr', 'asr', 'audi

In [12]:
video_title = video_metadata["title"]

print(f'Título do vídeo:\n\t"{video_title}"')
video_title = remove_special_characters(video_title).lower().replace(" ","-")
print(f'Formated label:\n\t"{video_title}"')

chapters = [c["title"] for c in video_metadata["chapters"]]

chapters

Título do vídeo:
	"NLP Demystified 14: Machine Translation With Sequence-to-Sequence and Attention"
Formated label:
	"nlp-demystified-14-machine-translation-with-sequencetosequence-and-attention"


['Seq2Seq and Attention',
 'Seq2Seq as a general problem-solving approach',
 'Translating language with a seq2seq model',
 'Machine translation challenges',
 'Effective decoding with Beam Search',
 'Evaluating translation models with BLEU',
 'The information bottleneck',
 'Overcoming the bottleneck with Attention',
 'Additive vs Multiplicative Attention',
 '[DEMO] Neural Machine Translation WITHOUT Attention',
 '[DEMO] Neural Machine Translation WITH Attention',
 'Attention as information retrieval']

#### Download do vídeo

In [6]:
%%time

ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
    'outtmpl': f'data/{video_title}.%(ext)s',
}

with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download([VIDEO_URL])

[youtube] Extracting URL: https://www.youtube.com/watch?v=tvIzBouq6lk
[youtube] tvIzBouq6lk: Downloading webpage
[youtube] tvIzBouq6lk: Downloading ios player API JSON
[youtube] tvIzBouq6lk: Downloading web creator player API JSON
[youtube] tvIzBouq6lk: Downloading m3u8 information
[info] tvIzBouq6lk: Downloading 1 format(s): 251
[download] Destination: data/nlp-demystified-14-machine-translation-with-sequencetosequence-and-attention.webm
[download] 100% of   62.43MiB in 00:00:26 at 2.33MiB/s     
[ExtractAudio] Destination: data/nlp-demystified-14-machine-translation-with-sequencetosequence-and-attention.mp3
Deleting original file data/nlp-demystified-14-machine-translation-with-sequencetosequence-and-attention.webm (pass -k to keep)
CPU times: user 1.84 s, sys: 365 ms, total: 2.2 s
Wall time: 1min 32s


# Text-to-Speech

In [8]:
print(whisper.available_models())

['tiny.en', 'tiny', 'base.en', 'base', 'small.en', 'small', 'medium.en', 'medium', 'large-v1', 'large-v2', 'large-v3', 'large']


In [9]:
%%time

model = whisper.load_model("large").to(device)

CPU times: user 22.3 s, sys: 3.27 s, total: 25.6 s
Wall time: 17.3 s


In [10]:
%%time

audio_filename = f"data/{video_title}.mp3"
raw_text_filename = f"data/raw_{video_title}.txt"


result = model.transcribe(audio_filename, language=TRANSCRIPTION_LANGUAGE)

with open(raw_text_filename, "w") as f:
    f.write(result["text"])

CPU times: user 14min 6s, sys: 3.32 s, total: 14min 9s
Wall time: 14min


In [12]:
import gc

try:
    del model
except:
    pass
    
gc.collect()

torch.cuda.empty_cache()

# Sintetizando o texto

In [36]:
%%time
from tqdm import tqdm
from langchain.prompts import PromptTemplate
from pathlib import Path
import tiktoken
from langchain.callbacks import get_openai_callback
from langchain_openai import ChatOpenAI

LLM_MODEL = "gpt-4o-mini"
INPUT_PRICE = 0.150 # per million
OUTPUT_PRICE = 0.600 # per million


raw_text_filename = f"data/raw_{video_title}.txt"
summary_directory = f"data/summary-{video_title}"

Path(summary_directory).mkdir(parents=True, exist_ok=True)

text_template = (
    "Considere o seguinte texto:\n"
    "<text>{text}</text>\n"
    f"Ele tem os seguintes capítulos: {str(chapters)}. \n"
    "Sua tarefa: Gere um texto didático em markdown que explique em detalhes o capítulo `{chapter}`.\n"
    "Instruções:"
    "1. A hierarquia máxima de títulos no markdown deve ser `##`.\n"
    "2. Retorne apenas o código markdown.\n"
    "3. Não retorne os delimitadores de bloco de código.\n"
    "4. Não economize nas explicações"
)

prompt = PromptTemplate(input_variables=["text", "chapter"], template=text_template)
llm = ChatOpenAI(temperature=0, model=LLM_MODEL)

chain = prompt | llm

CPU times: user 43.6 ms, sys: 2.13 ms, total: 45.8 ms
Wall time: 45.1 ms


In [39]:
from tqdm import tqdm

with open(raw_text_filename, "r") as f:
    text = f.read()

cost = 0

with tqdm(total=len(chapters)) as pbar:
    pbar.set_description(f"Custo = {round(cost*6, 4)} | Percentual")
    
    for i, chapter in enumerate(chapters):
        md_filename = f"{summary_directory}/{str(i+1).zfill(2)}-{chapter}.md"
        with get_openai_callback() as cb:
            summary = chain.invoke({"text": text, "chapter": chapter})
            cost += cb.total_cost

        md_content = f"# {i+1} - {chapter}\n\n{summary.content}"
        with open(md_filename, "w") as f:
            f.write(md_content)
        
        pbar.set_description(f"Custo = {round(cost*6, 4)} | Percentual")
        pbar.update(1)
    

Custo = 0.199 | Percentual: 100%|█████████████████████████| 12/12 [02:20<00:00, 11.75s/it]


In [47]:
md_content = [video_metadata["title"]]
for i, chapter in enumerate(chapters):
    md_chapter_filename = f"{summary_directory}/{str(i+1).zfill(2)}-{chapter}.md"
    with open(md_chapter_filename, "r") as f:
        chapter_content = f.read()
    md_content.append(chapter_content)

full_summary_filepath = f"{summary_directory}/complete-summary.md"
with open(full_summary_filepath, "w") as f:
    f.write("\n\n".join(md_content))

In [48]:
full_summary_filepath

'data/summary-nlp-demystified-14-machine-translation-with-sequencetosequence-and-attention/complete-summary.md'

# Traduzindo o texto


In [13]:
import tiktoken
from langchain.callbacks import get_openai_callback
from langchain_openai import ChatOpenAI

LLM_MODEL = "gpt-4o-mini"
INPUT_PRICE = 0.150 # per million
OUTPUT_PRICE = 0.600 # per million

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.text import TextLoader

book = TextLoader(raw_text_filename).load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000, 
    chunk_overlap=0, 
    separators=[". "],
    keep_separator=False,
)
paragraphs = text_splitter.split_documents(book)

In [105]:
%%time
from tqdm import tqdm
from langchain.prompts import PromptTemplate
from pathlib import Path

Path(f"data/translate-{video_title}").mkdir(parents=True, exist_ok=True)

text_template = (
    "Traduza o seguinte para português, mantendo os jargões técnicos em inglês. Não retorne nada exceto a tradução:\n"

    "{text}"

)

prompt = PromptTemplate(input_variables=["text"], template=text_template)
llm = ChatOpenAI(temperature=0, model=LLM_MODEL)

chain = prompt | llm

cost = 0
with tqdm(total=len(paragraphs)) as pbar:
    pbar.set_description(f"Custo = {round(cost*6, 4)} | Percentual")
    for i, p in enumerate(paragraphs):
        with get_openai_callback() as cb:
            summary = chain.invoke({"text": p.page_content+". "})
            cost += cb.total_cost
    
        with open(f"data/translate-{video_title}/{str(i).zfill(2)}.txt", "w") as f:
            f.write(summary.content)
        
        pbar.set_description(f"Custo = {round(cost*6, 4)} | Percentual")
        pbar.update(1)

Custo = 0.0686 | Percentual: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [02:21<00:00,  4.05s/it]

CPU times: user 799 ms, sys: 299 ms, total: 1.1 s
Wall time: 2min 21s


In [106]:
text_pieces = []
for i, p in enumerate(paragraphs):
    with open(f"data/translate-{video_title}/{str(i).zfill(2)}.txt", "r") as f:
        text_pieces.append(f.read())

text = " ".join(text_pieces)

with open(f"data/translated-{video_title}.txt", "w") as f:
    f.write(text)


# Gerando Questões

In [ ]:
import gc

try:
    del model
except:
    pass
    
gc.collect()

torch.cuda.empty_cache()

In [114]:
class LanguageModel:

    def __init__(self, tokenizer, model, device):
        self.tokenizer = tokenizer
        self.model = model
        self.device = device
        if self.tokenizer.pad_token_id is None:
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

    def tokenize(self, messages):
        text = tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(self.device)
        return model_inputs

    def generate(self, messages, max_new_tokens=2000, **kwargs):
        model_inputs = self.tokenize(messages)
        model_inputs["attention_mask"] = model_inputs["attention_mask"].to(
            model_inputs["input_ids"].device
        )
        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            attention_mask=model_inputs["attention_mask"],
            pad_token_id=self.tokenizer.pad_token_id,
            **kwargs
        )
        generated_ids = [
            output_ids[len(input_ids) :]
            for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [115]:
compute_dtype = torch.float16
attn_implementation = "flash_attention_2"

In [122]:
from transformers import AutoModelForCausalLM, AutoTokenizer

login(token=os.environ["HUGGINGFACE_TOKEN"])

model_id = "emdemor/question-generator-v2"
commit_hash = None


# A quantização é uma técnica para reduzir o tamanho do modelo e aumentar a eficiência computacional.
# Utilizamos a classe BitsAndBytesConfig para configurar a quantização em 4 bits, o que reduz o uso de memória e acelera o treinamento.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)

# Usamos a classe AutoModelForCausalLM para carregar um modelo pré-treinado adequado para modelagem de linguagem causal.
# Parâmetros importantes incluem:
#  - torch_dtype=compute_dtype: Define o tipo de dado para o modelo.
#  - quantization_config=bnb_config: Aplica a configuração de quantização.
#  - device_map="auto": Distribui automaticamente o modelo nos dispositivos disponíveis.
#  - attn_implementation=attn_implementation: Define a implementação da atenção.
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=compute_dtype,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation,
    revision=commit_hash,
)

# # adapta o modelo para o treinamento em k-bits, otimizando ainda mais o desempenho.
# model = prepare_model_for_kbit_training(model)


def set_tokenizer(model_id):
    tokenizer = AutoTokenizer.from_pretrained(model_id, revision=commit_hash)
    tokenizer.padding_side = "right"
    return tokenizer


tokenizer = set_tokenizer(model_id)

llm = LanguageModel(tokenizer, model, device="cuda")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [124]:
def generate_question(context, temperature=0.01):
    return llm.generate([{"content": f"{context}", "role": "user"}], temperature=temperature)

In [127]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.text import TextLoader


book = TextLoader(f"data/translated-{video_title}.txt").load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000, 
    chunk_overlap=0, 
    separators=[". "],
    keep_separator=False,
)
paragraphs = text_splitter.split_documents(book)

In [199]:
%%time
import json
from json import JSONDecodeError
from tqdm.notebook import tqdm
import pandas as pd
import json
import re
csv_filename = f"data/questions-{video_title}.csv"
pd.DataFrame().to_csv(csv_filename)

print("Endereço =",csv_filename)



def correct_json(json_string):
    corrected_json_string = json_string
    if json_string[-2:] in ["]]", "]}"]:
        corrected_json_string  = corrected_json_string[:-1]
    return corrected_json_string


def generate(p):
    temperature = 0.01

    for i in range(10):
        try:     
            qa_pairs_string = generate_question(p.page_content, temperature=temperature)
            qa_pairs_string = correct_json(qa_pairs_string)
            qa_pairs = json.loads(qa_pairs_string)
            base_df = pd.read_csv(csv_filename)
            updated_df = pd.concat([base_df, pd.DataFrame(qa_pairs)])
            updated_df.to_csv(csv_filename, index=False)
            return
        except JSONDecodeError as err:
            temperature += 0.05
            continue


for p in tqdm(paragraphs, total=len(paragraphs)):
    generate(p)

Endereço = data/questions-nlp-demystified-13-recurrent-neural-networks-and-language-models.csv


  0%|          | 0/37 [00:00<?, ?it/s]

CPU times: user 23min 9s, sys: 735 ms, total: 23min 10s
Wall time: 23min 9s
